In [1]:
import numpy as np
import pandas as pd

In [2]:
def get_precip_data():
    return pd.read_csv('precipitation.csv', parse_dates=[2])

In [3]:
def date_to_month(year, month):
    # You may need to modify this function, depending on your data types.
    return '%04i-%02i' % (year, month)

In [6]:
def pivot_months_pandas(data):
    """
    Create monthly precipitation totals for each station in the data set.
    
    This should use Pandas methods to manipulate the data.
    """
    month = data['date'].dt.month
    year = data['date'].dt.year
    
    date_month = np.vectorize(date_to_month)
    data['month'] = date_month(year,month)
    
    monthly_num = data.groupby(['name','month']).aggregate('sum').reset_index()
    monthly = monthly_num.pivot(index='name', columns='month', values='precipitation')
    
    count_num = data.groupby(['name','month']).aggregate('count').reset_index()
    counts = count_num.pivot(index='name', columns='month', values='precipitation')
    # ...
    print(monthly_num)
    return monthly, counts

In [7]:
def pivot_months_loops(data):
    """
    Create monthly precipitation totals for each station in the data set.
    
    This does it the hard way: using Pandas as a dumb data store, and iterating in Python.
    """
    # Find all stations and months in the data set.
    stations = set()
    months = set()
    for i,r in data.iterrows():
        stations.add(r['name'])
        m = date_to_month(r['date'])
        months.add(m)

    # Aggregate into dictionaries so we can look up later.
    stations = sorted(list(stations))
    row_to_station = dict(enumerate(stations))
    station_to_row = {s: i for i,s in row_to_station.items()}
    
    months = sorted(list(months))
    col_to_month = dict(enumerate(months))
    month_to_col = {m: i for i,m in col_to_month.items()}

    # Create arrays for the data, and fill them.
    precip_total = np.zeros((len(row_to_station), 12), dtype=np.uint)
    obs_count = np.zeros((len(row_to_station), 12), dtype=np.uint)

    for _, row in data.iterrows():
        m = date_to_month(row['date'])
        r = station_to_row[row['name']]
        c = month_to_col[m]

        precip_total[r, c] += row['precipitation']
        obs_count[r, c] += 1

    # Build the DataFrames we needed all along (tidying up the index names while we're at it).
    totals = pd.DataFrame(
        data=precip_total,
        index=stations,
        columns=months,
    )
    totals.index.name = 'name'
    totals.columns.name = 'month'
    
    counts = pd.DataFrame(
        data=obs_count,
        index=stations,
        columns=months,
    )
    counts.index.name = 'name'
    counts.columns.name = 'month'
    
    return totals, counts


def main():
    data = get_precip_data()
    totals, counts = pivot_months_pandas(data)
    totals.to_csv('totals.csv')
    counts.to_csv('counts.csv')
    np.savez('monthdata.npz', totals=totals.values, counts=counts.values)

if __name__ == '__main__':
    main()

                               name    month   latitude  longitude  elevation  \
0            BURNABY SIMON FRASER U  2016-01  1478.4990 -3687.5010    10980.0   
1            BURNABY SIMON FRASER U  2016-02  1084.2326 -2704.1674     8052.0   
2            BURNABY SIMON FRASER U  2016-03  1478.4990 -3687.5010    10980.0   
3            BURNABY SIMON FRASER U  2016-04  1478.4990 -3687.5010    10980.0   
4            BURNABY SIMON FRASER U  2016-05  1034.9493 -2581.2507     7686.0   
5            BURNABY SIMON FRASER U  2016-06  1034.9493 -2581.2507     7686.0   
6            BURNABY SIMON FRASER U  2016-07   985.6660 -2458.3340     7320.0   
7            BURNABY SIMON FRASER U  2016-08  1232.0825 -3072.9175     9150.0   
8            BURNABY SIMON FRASER U  2016-09   985.6660 -2458.3340     7320.0   
9            BURNABY SIMON FRASER U  2016-10   936.3827 -2335.4173     6954.0   
10           BURNABY SIMON FRASER U  2016-11   837.8161 -2089.5839     6222.0   
11           BURNABY SIMON F